1. Добиться хорошего результата от модели, собранной на занятии (5 угаданныx картинок из 8 предложенных). Варианты изменений:<br>
изменение слоёв и их количества;<br>
изменение метода оптимизации;<br>
изменение процесса обучения;<br>
*преобразование данных transform<br>
2.*Переписать данный туториал на PyTorch: https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ru

In [ ]:
import numpy as np
import pandas as pd

import torch
from torch import optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn

import matplotlib.pyplot as plt

from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

__Ниже перепишу часть кода с урока и непосредственно саму модель, которую нужно улучшить:__

In [ ]:
# ЗАГРУЗКА ТРЕЙНА И ТЕСТА
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True,  # можно таким же образом скачать тестовую часть
                                             transform=transforms.ToTensor(), #чтобы сразу преобразовывалось в корректный тип и тут нормировка происходит сразу 
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transforms.ToTensor())


image, label = train_dataset[0]
print(image.size())
print(label)

In [ ]:
# plt требует классического вида shape, потому преобразовали через пермут

plt.imshow(image.permute(1, 2, 0).numpy()); # 1 2 0 - это индексы изначальноых параметров image

In [ ]:
# НАСТРОЙКА ПОДАЧИ КАРТИНОК С ТЕСТА  ТРЕЙНА В DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, # какой кусок объектов выдавать. 64 картинки
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8,
                                          shuffle=False)

In [ ]:
# НАЗВАНИЯ КЛАССОВ ДАТАСЕТА - ВСЕГО 10 КЛАССОВ
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# МОДЕЛЬ С УРОКА
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)     # инициализиурем 4 слоя
        self.fc2 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1) # вытягиваем в одномерный вектор - аналог flatten
        x = self.fc1(x)      # прононяем по первому слою
        x = F.leaky_relu(x)  # применяем функцию активации лики релу
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.fc4(x)      # на четвертом слое не сделали ф-цию активации, тут многоклассовость и д.б. софт макс, мы ее в предсказании сделаем
        return x             # в прямом проходе всегда следует останавливаться на том, что идет в ф-цию активации в конце и не писать ее,т.к. ф-ции потерь считаются в торче
                             # на логитах, а логиты это то что идет в ф-цию активации
    
    def predict(self, x):            # тот же форвард, но с софтмаксов на конце (выделяет 1 класс из логитов)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x) 
        x = F.leaky_relu(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.fc4(x)
        x = F.softmax(x)
        return x


net = Net(3072, 100, 10)  # 3072 это кол-во каналов при входе усноженное на размерность картинки: 3 * 32 * 32
net.train()

criterion = nn.CrossEntropyLoss()  # ф-ция потерь в торче использует именно логиты. получается что с forward поступают логиты и она их считывает, поэтому нет в конце активации выше
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.0) # net.parameters() это мы так указываем подгружая параметры модели

num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    running_items = 0.0


    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

         # Обнуляем градиент
        optimizer.zero_grad()
        # Делаем предсказание
        outputs = net(inputs)
        # Рассчитываем лосс-функцию
        loss = criterion(outputs, labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}')
            running_loss, running_items = 0.0, 0.0

print('Training is finished!')

In [ ]:
# ДАЛЕЕ КАЧЕСТВО РАБОТЫ МОДЕЛИ -СМОТРИМ МЕТРИКУ И ПРЕДМЕСТНО КАК УГАДЫВАЕТ КАРТИНКИ
data_iter = iter(test_loader)
images, labels = next(data_iter)

In [ ]:
net.eval()
outputs = net(images)  # говорит что net(images) это равнозначно net.forward(images) типа можно упрощать 
imgs = torchvision.utils.make_grid(images)
plt.figure(figsize=(10, 5))
plt.imshow(imgs.permute(1, 2, 0).numpy())
print('GroundTruth: ', ' '.join(classes[labels[j]] for j in range(len(labels))))

In [ ]:
_, predicted = torch.max(outputs, 1)

predicted

In [ ]:
gt = np.array([classes[labels[j]] for j in range(len(labels))])
pred = np.array([classes[predicted[j]] for j in range(len(labels))])

print(gt)
print(pred)
print(f'Accuracy is {(gt == pred).sum() / len(gt)}')

__РЕШЕНИЕ:__<br>
Cначала проверяем предоставляет ли в данный момент колаб возможность использования GPU.
<br>

если да, то в цикле ниже и модель и все inputs, labels (трейн и предсказание)нужно положить на GPU через метод .to_device(device)

На этапе выполнения 72 итераций ниже я подключил себе версию про на колабе и имел доступ к видеокарте НВИДИА с 40гб. памяти, в гугл колабе на странице данной работы в рантайме включил GPU и потом на gpu клал саму тестируемую модель нейросети и inputs & labels (все входы в нее), однако на скорость обучения это не повлияло - увидим ниже.

In [ ]:
print(f"Поддерживается ли CUDA : {torch.cuda.is_available()}")
print(f'Количество гпу девайсов: {torch.cuda.device_count()}')
print(f"Характеристики видеокарты : {torch.cuda.get_device_properties(0)}")
print(f"Удаляем всю незанятую память через torch.cuda.empty_cache()")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

__Я решил одновременно прогонять по циклу и смотреть качество работы моделей по следующим параметрам:__
* кол-во скрытых слоев нейросети (для этого ниже настроил универсальный способ создания сети),
* кол-во нейронов,
* кол-во эпох обучения каждой сети,
* скорость обучения оптимизатора.

In [ ]:
# МОЯ ХИТРО НАПИСАННАЯ МОДЕЛЬ С ВОЗМОЖНОСТЬЮ АВТОМАТИЧЕСКОГО СОЗДАНИЯ СКРЫТЫХ СЛОЕВ СКОЛЬКО ХОЧЕШЬ 
class My_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers=3):
        super().__init__()
        self.n_layers = n_layers
        self.layers = nn.ModuleList()
        
        # ниже примерная логика как можно изменять кол-во нейронов от слоя к слою, чтобы выходные из предыдущего равнялись кол-ву входных следующего слоя
        self.layers.append(nn.Linear(input_dim, (2 ** (n_layers - 2)) * hidden_dim)) # первый слой вне цикла, т.к. инпуты тут фиксированные, а в цикле будем подбирать
        for i in range(1, n_layers - 1): #  ВООБЩЕ ХОТЕЛОСЬ БЫ чтобы  n_layers отображал именно кол-во скрытых слоев, а не всего слоев
          self.layers.append(nn.Linear((2 ** (n_layers - i - 1)) * hidden_dim, (2 ** (n_layers - i - 2)) * hidden_dim))
        self.layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x):
        x = x.view(x.shape[0], -1)         # вытягиваем в одномерный вектор - аналог flatten
        for layer in self.layers[:-1]: # последний не берем, т.к. для него не нужна активация
          x = layer(x)             # прононяем по слою
          x = F.leaky_relu(x)      # применяем функцию активации лики релу
        x = self.layers[-1](x)
        return x
  


__Далее будет долгий цикл и чтобы колаб не спрашивал подверждения что я работ при ожидании обучения нейросети в не активном режиме (я ушел от компа), настроил кликер в javascript браузера (в консоли инспектора)__

<br>
Steps <br>
    Сначала создать пустую строку ввода кода - туда нажать курсор и далее по схеме:<br>
    Open the inspector view by typing Ctrl+ Shift + i and then clicking on console tab at top.
    Paste the below code snippet at bottom of console and hit enter


```
function ClickConnect(){
console.log("Working"); 
document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
setInterval(ClickConnect,60000)
```



__Ниже идет тот самый долгий цикл, где я перебираю каждую модель и после последней эпохи записываю параметры, в т.ч. время обучения в секундах. Решил проблему обнуления весов у одной и той же модели, но где перебирается разное кол-во эпох. Все вроде бы корректно.__

<br>

__Примечание по рассчету кросс-энтропии:__ Похоже что, встроенная функция расчитыват усредненный лосс на батче. Поэтому надо в конце эпохи делить аккумулятивный лосс на количество батчей, а не на количество картинок. Именно поэтому лоссы на тесте и на трейне на порядок не сходились в методичке

In [ ]:
import time 

criterion = nn.CrossEntropyLoss()  

num_layers = range(3, 7) # мой класс нейросети Net написан так, что число отсюда это кол-во всего слоев, чтобы увидеть именно число скрытых слоев, нужно отнять 2 (входной, выходной)
n_dims = [10, 30, 50] # в модели нейросети на разных слоях кол-во нейронов будет исчисляться путем умножения взятого числа из n_dims на число от 2 до 8
num_epochs = [5,10,20]
lr_rate = [0.0001, 0.001]

# сюда буду записывать параметры каждой модели
results = pd.DataFrame(columns=['hidden dim', 'learning rate','epochs', 'time (sec)','train loss','test loss', 'train acc', 'test acc'])

count = 1
for l in num_layers:
  for dim in n_dims:
    torch.cuda.empty_cache()
    net_to_examine = Net(input_dim=3072, hidden_dim=dim, output_dim=10, n_layers=l).to(device)  # 3072 размерность картинки * входы: 32*32*3, а 10 - кол-во классов

    for rate in lr_rate:
      optimizer = optim.Adam(net_to_examine.parameters(), lr=rate)

      for n in num_epochs:
        start = time.time() # буду сравнивать время обучения каждой модели

        # включаем обучение
        net_to_examine.train()

        for epoch in range(n):
            running_loss = 0.0
            running_items = 0.0
            running_loss_test = 0.0
            running_items_test = 0.0
            running_right = 0.0
            running_right_test = 0.0

            #net_to_examine.train() сли тут, то последние веса прошлой модели идут в первую эпоху следующей модели
            for i, data in enumerate(train_loader):
                
                inputs, labels = data[0].to(device), data[1].to(device)

                # Обнуляем градиент внутри каждой эпохи итерируемых эпох 1 модели
                optimizer.zero_grad()
                # Делаем предсказание
                outputs = net_to_examine(inputs)
                # Рассчитываем лосс-функцию
                loss = criterion(outputs, labels)
                # Делаем шаг назад по лоссу
                loss.backward()
                # Делаем шаг нашего оптимайзера
                optimizer.step()

                # выводим статистику о процессе обучения
                running_loss += loss.item()
                running_items += len(labels)
                running_right += (labels == torch.max(outputs, 1)[1]).sum()

            running_right = running_right.item()
            train_loss = running_loss / i
            train_acc = running_right / running_items
            print(f'Epoch [{epoch + 1}/{n}]. ' \
                  f'Loss_train: {train_loss:.4f}. ' \
                  f'Acc: {train_acc:.4f}.', end=' ')
            

            net_to_examine.eval()
            for i, data in enumerate(test_loader):
                inputs, labels = data[0].to(device), data[1].to(device)
                # Делаем предсказание
                outputs = net_to_examine(inputs)
                # Рассчитываем лосс-функцию
                loss = criterion(outputs, labels)

                running_loss_test += loss.item()
                running_items_test += len(labels)
                running_right_test += (labels == torch.max(outputs, 1)[1]).sum()

            running_right_test = running_right_test.item()
            test_loss = running_loss_test / i
            test_acc = running_right_test / running_items_test
            print(f'Loss_test: {test_loss:.4f}. ' \
                  f'Acc_test: {test_acc:.4f}')


            # итерация одной модели со всеми проверяемыми параметрами заканчивается после отработки последней эпохи и именно тут записыываем скор и считаем время
            if (epoch +1) == n:
              end = time.time() - start
              results = results.append({'hidden layers': l-2,  # считать буду только скрытые слои, т.к. именно эта инф. полезна - зачем учитывать входной и выходной слой
                              'hidden dim': dim,
                              'learning rate': rate,
                              'epochs': n,
                              'time (sec)': int(end),
                              'train loss': train_loss,
                              'test loss': test_loss,
                              'train acc': train_acc, 
                              'test acc': test_acc}, ignore_index=True)
              
              # чтобы понимать сколько еще осталось ждать до окончания цикла
              print(f'Завершилось обучение модели № {count} из {len(n_dims)*len(num_epochs)*len(lr_rate)*len(num_layers)}\n')
              count += 1

              if count % 24 == 0:
                results.to_csv(f'Grid_Search_result 1 - {count}.csv', index = False)
                files.download('Grid_Search_result 1 - {count}.csv')

              #перезаписывая модель заново таким образом обнуляю веса с последней эпохи  перед включением трейна обратно
              net_to_examine = Net(input_dim=3072, hidden_dim=dim, output_dim=10, n_layers=l).to(device)
              optimizer = optim.Adam(net_to_examine.parameters(), lr=rate)

            # независимо последняя эпоха или нет, после каждой эпохи обнуляем значения для подсчета метрик
            running_loss_test, running_items_test, running_right_test = 0.0, 0.0, 0.0
            running_loss, running_items, running_right = 0.0, 0.0, 0.0


print('Training is finished!')

results.to_csv('Grid_Search_result FINAL.csv', index = False)
files.download('Grid_Search_result FINAL.csv')

__Я скачивал в цикле данные и по итогам цикла и далее я сейчас погружу весь результат работы цикла (чтобы заново не запускать цикл):__

In [ ]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/Grid_Search_result FINAL.csv', "rb") as f:
    results = pd.read_csv(f)
    
results

In [ ]:
# т.к. время в секундах, то хочу веревести в минуты
results['time (sec)'] = results['time (sec)'].apply(lambda x: float(str(f'{x / 60: .2f}')))

In [ ]:
results

In [ ]:
# чтобы название теперь соответствовало цифрам перепишу его - можно было сразу сделать в цикле кстати
results.rename(columns={'time (sec)': 'time (min)'}, inplace=true)
results.head(3)

In [ ]:
results['time (sec)'].sum() / 60

### Цикл работал 3.5 часа!

In [ ]:
# сортирую датафрейм так чтобы сверху был лучший скор и одновременно меньшее время обучения
results.sort_values(['test acc','time (sec)'], ascending=[False, True]).head(9)

Видим что победила 68 модель (индекс здесь соответствует номеру итерации в цикле и сотв-но номеру модели).

<br> 

Создам теперь модель именно с этими параметрами и буду ожидать от нее 52% точности на тесте:

In [ ]:
final_net = My_Net(input_dim=3072, hidden_dim=50, output_dim=10, n_layers=6) # n_layers указываю на 2 больше, т.к. 4 в таблице это именно
                                                                                       # скрытые слои, т.е. без учета входного и выходного слоя
criterion = nn.CrossEntropyLoss()
final_optimizer = optim.Adam(final_net.parameters(), lr=0.0001)    
epochs = 20

# обучения
for epoch in range(epochs):
    final_net.train() # поскольку у меня в цикле обучения внедрена часть кода проверки предсказания на тесте, то в предсказнии обучени отключаю, а перед обучением надо включать

    running_loss = 0.0
    running_items = 0.0
    running_loss_test = 0.0
    running_items_test = 0.0
    running_right = 0.0
    running_right_test = 0.0

    #net_to_examine.train() сли тут, то последние веса прошлой модели идут в первую эпоху следующей модели
    for i, data in enumerate(train_loader):
        
        inputs, labels = data[0], data[1]

        # Обнуляем градиент внутри каждой эпохи итерируемых эпох 
        final_optimizer.zero_grad()
        # Делаем предсказание
        outputs = final_net(inputs)
        # Рассчитываем лосс-функцию
        loss = criterion(outputs, labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        final_optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()

    running_right = running_right.item()
    train_loss = running_loss / i
    train_acc = running_right / running_items
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
          f'Loss_train: {train_loss:.4f}. ' \
          f'Acc: {train_acc:.4f}.', end=' ')
    

    final_net.eval()
    for i, data in enumerate(test_loader):
        inputs, labels = data[0], data[1]
        # Делаем предсказание
        outputs = final_net(inputs)
        # Рассчитываем лосс-функцию
        loss = criterion(outputs, labels)

        running_loss_test += loss.item()
        running_items_test += len(labels)
        running_right_test += (labels == torch.max(outputs, 1)[1]).sum()

    running_right_test = running_right_test.item()
    test_loss = running_loss_test / i
    test_acc = running_right_test / running_items_test
    print(f'Loss_test: {test_loss:.4f}. ' \
          f'Acc_test: {test_acc:.4f}')


    # независимо последняя эпоха или нет, после каждой эпохи обнуляем значения для подсчета метрик
    running_loss_test, running_items_test, running_right_test = 0.0, 0.0, 0.0
    running_loss, running_items, running_right = 0.0, 0.0, 0.0

__Финальная часть - смотрим как распознает картинки на примере:__

In [ ]:
data_iter = iter(test_loader)
images, labels = next(data_iter)

final_net.eval()
outputs = final_net(images)  # final_net(images) это равнозначно final_net.forward(images) - можно упрощать 
imgs = torchvision.utils.make_grid(images)
plt.figure(figsize=(10, 5))
plt.imshow(imgs.permute(1, 2, 0).numpy())
print('GroundTruth: ', ' '.join(classes[labels[j]] for j in range(len(labels))))

In [ ]:
_, predicted = torch.max(outputs, 1)

predicted

In [ ]:
gt = np.array([classes[labels[j]] for j in range(len(labels))])
pred = np.array([classes[predicted[j]] for j in range(len(labels))])

print(gt)
print(pred)
print(f'Accuracy is {(gt == pred).sum() / len(gt)}')

Итак, модель распознала 5 из 8.

Попробуем сделать то же самое, но без деления на батчи в критерии (делаем как на уроке было):

In [ ]:
final_net2 = My_Net(input_dim=3072, hidden_dim=50, output_dim=10, n_layers=6) # n_layers указываю на 2 больше, т.к. 4 в таблице это именно
                                                                                       # скрытые слои, т.е. без учета входного и выходного слоя
criterion = nn.CrossEntropyLoss()
final_optimizer = optim.Adam(final_net2.parameters(), lr=0.0001)    

num_epochs = 20  

for epoch in range(num_epochs):
    running_loss = 0.0
    running_items = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]
        
        # Обнуляем градиент
        final_optimizer.zero_grad()
        # Делаем предсказание
        outputs = final_net2(inputs)
        # Рассчитываем лосс-функцию
        loss = criterion(outputs, labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        final_optimizer.step()
        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        if i % 300 ==0:  # печатаем каждые 300 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}')
            running_loss, running_items = 0.0, 0.0
            
print('Training is finished!')



In [ ]:
data_iter = iter(test_loader)
images, labels = next(data_iter)

final_net2.eval()
outputs = final_net2(images)  # final_net(images) это равнозначно final_net.forward(images) - можно упрощать 
imgs = torchvision.utils.make_grid(images)
plt.figure(figsize=(10, 5))
plt.imshow(imgs.permute(1, 2, 0).numpy())
print('GroundTruth: ', ' '.join(classes[labels[j]] for j in range(len(labels))))

In [ ]:
_, predicted = torch.max(outputs, 1)

gt = np.array([classes[labels[j]] for j in range(len(labels))])
pred = np.array([classes[predicted[j]] for j in range(len(labels))])

print(gt)
print(pred)
print(f'Number of matches is {(gt == pred).sum()}')
print(f'Accuracy is {(gt == pred).sum() / len(gt)}')